In [49]:
from pathlib import Path
import csv
import json
from typing import NamedTuple, Dict, List, Tuple
from pprint import pprint
from operator import itemgetter
import re
from itertools import chain
from collections import defaultdict
from math import ceil
import os

import pandas as pd


DATASET = Path('/data/NER/VectorX/annotated/assignments_Разметка+ORG+2017-08-31_01-06-2018.tsv')
NE5Format = Path('/data/NER/VectorX/NE5_test')
# DATASET = Path('/data/NER/VectorX/annotated/assignments_Разметка+ORG+2017-08-01_08-06-2018.tsv')
# NE5Format = Path('/data/NER/VectorX/NE5_train')
if not os.path.isdir(NE5Format):
    os.mkdir(NE5Format)

san_dquoter = re.compile(r"(«|»|“|”|``|'|„|“)")

In [50]:
data = pd.read_csv(str(DATASET), delimiter='\t')

In [51]:
class EntityCandidate():
    start: int
    end: int # included
    options: Dict[Tuple[int], List[str]]
    
    def __init__(self, worker_id, start_index, end_index):
        self.start = start_index
        self.end = end_index
        self.options = {}
        self.add_option(worker_id, self.start, self.end)
        
    def is_same_entity(self, start_index, end_index):
        return (self.start <= start_index <= self.end) or \
               (self.start <= end_index <=self.end)
          
    def add_option(self, worker_id, start_index, end_index):
        """mutable function"""
        self.start = min(self.start, start_index)
        self.end = max(self.end, end_index)
        span = start_index, end_index
        if span not in self.options:
            self.options[span] = []
        self.options[span].append(worker_id)

In [52]:
def get_spans(orig, words, tags):
    idx = 0
    last_word_end = -1
    span_start = -1
    span_end = -1
    spans = []
    for w, t in zip(words, tags):    
        while orig[idx] == ' ':
            idx += 1

        assert orig[idx:].startswith(w), [orig[idx:idx+10], w]

        if not w.strip():
            idx += len(w)
            continue

        if span_start != -1 and not t.startswith('I'):
            ent_candidate = orig[span_start:last_word_end]
            beg = re.match(r'^" *', ent_candidate)
            end = re.search(r' *"$', ent_candidate)
            
            if beg and end:
                print('!!!!!', ent_candidate)
                left_offset = beg.span()[1]
                ent_candidate = ent_candidate[left_offset:]
                span_start += left_offset
 
                right_offset = end.span()[1] - end.span()[0]
                ent_candidate = ent_candidate[:-right_offset]
                last_word_end -= right_offset
                
            spans.append([span_start, last_word_end, ent_candidate])
            span_start = -1    

        if t.startswith('B'):
            span_start = idx

        idx += len(w)
        last_word_end = idx
    return spans

In [53]:
class Proportion:
    def __init__(self):
        self.positive_count = 0
        self.negative_count = 0
    def incr_positives(self, count=1):
        self.positive_count += count
    def incr_negatives(self, count=1):
        self.negative_count += count        
    def mean(self):
        return self.positive_count / (self.positive_count + self.negative_count)
    def harmonic_mean(self):
        return (self.positive_count - 1) / (self.positive_count + self.negative_count - 1)
    def __repr__(self):
        return f'{self.positive_count}/{self.positive_count + self.negative_count}'
    def __iadd__(self, other):
        assert isinstance(other, Proportion)
        self.positive_count += other.positive_count
        self.negative_count += other.negative_count
        return self
    def __add__(self, other):
        p = Proportion()
        p += self
        p += other
        return p
        
#     def __repr__(self):
#         return f'{self.positive_count}/{self.positive_count + self.negative_count}'

In [54]:
overall_recall = defaultdict(Proportion)
overall_precision = defaultdict(Proportion)
overall_correctness = defaultdict(Proportion)

for inp, d in data.groupby('INPUT:input'):
    orig = d['INPUT:orig'].iloc[0]
    nice_orig = san_dquoter.sub('"', orig).replace('\r\n', '\n')
    guid = d['INPUT:guid'].iloc[0]
    words = san_dquoter.sub('"', inp.replace('\r\n', '\n')).split(' ')
    tags = ['O' for _ in words]
    print(' '.join(words))

    entity_candidates = []
    for _, row in d.iterrows():
        idx = json.loads(row['OUTPUT:output'])
        wid = row['ASSIGNMENT:worker_id']
#         wid = row['ASSIGNMENT:link']

        
        for start, end in idx:
            for ec in entity_candidates:
                if ec.is_same_entity(start, end):
                    ec.add_option(wid, start, end)
                    break
            else:
                entity_candidates.append(EntityCandidate(wid, start, end))
                
    print()
    
    print('len(d)', len(d))
    workers = set(d['ASSIGNMENT:worker_id'])
#     workers = set(d['ASSIGNMENT:link'])

    assigment_recall = {w: Proportion() for w in workers}
    assigment_precision = {w: Proportion() for w in workers}
    assigment_correctness = {w: Proportion() for w in workers}
    
    workers_count = max(1, len(workers))
    for e in sorted(entity_candidates, key=lambda ec: ec.start):
        finders = set(chain(*e.options.values()))
        not_finders = workers - finders        
        entity_finders_count = len(finders)
        is_entity_real = entity_finders_count >= ceil(workers_count / 2)
        
        correct_span = None
        
        # put tags
        if is_entity_real:
            correct_span = max(e.options.items(), key=lambda x: len(x[1]))[0]
            
            s_id, e_id = correct_span
            tags[s_id] = 'B-ORG'
            for i in range(s_id+1, e_id+1):
                tags[i] = 'I-ORG'
                
        # calculate workers statistics
        if is_entity_real:
            for w in not_finders:
                assigment_recall[w].incr_negatives()
            for w in finders:
                assigment_recall[w].incr_positives()
                
            correct_finders = set(e.options[correct_span])
            for w in finders:
                if w in correct_finders:
                    assigment_correctness[w].incr_positives()
                else:
                    assigment_correctness[w].incr_negatives()                
        for w in finders:
            if is_entity_real:
                assigment_precision[w].incr_positives()
            else:
                assigment_precision[w].incr_negatives()                
            
#             print(' '.join(words[s: e+1]), span, f'{score:.2}')
#         for span, workers in e.options.items():
#             score = len(workers) / workers_count
#             s, e = span
#             print(' '.join(words[s: e+1]), span, f'{score:.2} ({len(workers)}/{workers_count})')
    for w, stats in assigment_recall.items():
        overall_recall[w] += stats
    for w, stats in assigment_precision.items():
        overall_precision[w] += stats
    for w, stats in assigment_correctness.items():
        overall_correctness[w] += stats

    
    spans = get_spans(nice_orig, words, tags)
    print(spans)
    print('recall', assigment_recall)
    print('precision', assigment_precision)
    print('correctness', assigment_correctness)
    

        
# 00001c1dcc--5b17b7a16502bd010bfae955        reject it

    with (NE5Format / f'{guid}.txt').open('w') as f:
        f.write(nice_orig)
        
    with (NE5Format / f'{guid}.ann').open('w') as f:
        w = csv.writer(f, delimiter='\t', quotechar='|')
        for i, span in enumerate(sorted(spans, key=itemgetter(0))):
            w.writerow([f'T{i+1}', f'ORG {span[0]} {span[1]}', nice_orig[span[0]:span[1]]])


 
 
 
 
 Камеры видеонаблюдения помогли найти квартирного воришку . Мужчина совершил 45 краж в Архангельске и Северодвинске . При этом , некоторые пострадавшие узнали о краже только от сотрудников полиции . Злоумышленник ходил по подъездам и проверял - в каких квартирах открыты двери . После этого , снимал верхнюю одежду , заходил в прихожую , брал ценности и сумочки . Вычислить подставного соседа смог один из жителей многоквартирного дома . С помощью камер в подъезде , он увидел подозрительного мужчину и вызвал полицию . 
 Ольга Яшунова , юрист правового отдела УМВД России по Архангельской области : 
 - Видеонаблюдение является большим помощником в расследовании преступлений . Для того , чтобы установить видеонаблюдение во дворе жилого дома , либо в помещениях жилых домов , необходимо собрать собрание собственников , на которых установить количество установленных камер и места их размещения . 
 Известно , что мужчина уже не раз был судим за кражи . Сейчас подозреваемый находится в СИ

correctness {'6ee417ba93c9fb336e6265c12f64e9d3': 0/0, 'ecf2f1d5973683b5171249a0463d594c': 0/0, 'a6d97932c543ba2fb32df13249795a18': 1/1, '4dc719d11a8695d46b2cc6973b27a2c4': 1/1, 'c1a2999343da2d84effa1679a4a000b6': 2/2, '258e20509b3e018e01dace2edf9a0c2d': 1/1, '572ee784bf7b64344b2ef28600e27918': 2/2, '5e4a3fd19550dbfa5b799208b2e57ed3': 1/1, '6703036b745a34776ee3a61e5c252bc9': 2/2}
Глава администрации Никольского района Людмила Линина приняла участие в отчете главы администрации Ильминского сельсовета Михаила Апаркина перед населением села Соколовка о проделанной работе в период с 2012 по 2016 годы и перспективах развития сельсовета . 
 В ходе доклада глава администрации Ильминского сельсовета доложил присутствующим о социально - экономическом развитии сельсовета , проводимой работе по энергосбережению , благоустройству территории и обеспечению мер противопожарной безопасности . 
 " В 2016 году мы установили детские площадки в селах Ильмино , Соколовка и Усть - Инза , благодаря программе 

Прокуратура г . Петрозаводска утвердила обвинительное заключение по уголовному делу в отношении 35 - летнего петрозаводчанина , обвиняемого в совершении мошеннических действий . 
 Органамим предварительного следствия ему инкриминируется совершение трех преступлений , предусмотренных ч . 3 и 4 ст . 159 Уголовного кодекса Российской Федерации ( мошенничество , совершенное в крупном и в особо крупном размере ) . 
 Согласно материалам уголовного дела , с 2013 года по 2016 год обвиняемый являлся руководителем пяти организаций , осуществляющих строительно - ремонтные работы . 
 Предприниматель заключал договоры с гражданами на осуществление строительных , ремонтно - отделочных , малярных и штукатурных работ . Однако , получив предоплату от заказчиков , ремонт он не заканчивал , либо не начинал вовсе . 
 В ходе предварительного следствия по делу было установлено 19 потерпевших , пострадавших от действий обвиняемого , причиненный им общий ущерб составил более 2 , 2 миллиона рублей . 
 Уголовно

In [55]:
overall_f1 = {}
for w in set(overall_precision.keys()).union(overall_recall.keys()):
    p = overall_precision[w].mean()
    r = overall_recall[w].mean()
#     overall_f1[w] = 2*p*r/(p+r)
    overall_f1[w] = overall_precision[w] + overall_recall[w]
sorted(overall_f1.items(), key=lambda x: x[1].harmonic_mean())

[('f8b9d394e4dfee0d03f5b7ed9c441c5e', 16/27),
 ('5e4a3fd19550dbfa5b799208b2e57ed3', 228/366),
 ('1d3106d5230c0d75014e69aecec5a8e2', 126/196),
 ('9c7674ffdd1021cdbfa7f252a628c0b1', 92/140),
 ('b87a0a77513b47131319bfbc471f191c', 50/72),
 ('39d10af932c65c9349293fc361cd784f', 100/143),
 ('e046228bcf79c8cbb840162708b2782e', 18/25),
 ('258e20509b3e018e01dace2edf9a0c2d', 556/771),
 ('aee84a090e5ba3074337454df6250066', 30/41),
 ('426b1589509eb52dfe8c9e501e598c11', 52/69),
 ('4d60fcffff02708feee58b1d693d231f', 40/53),
 ('0f99672e98a5a271ed426c3c3b3e3d10', 120/159),
 ('398fea4887eb0c5a5d2869da25e28b06', 48/63),
 ('8d533ef61a97428c2bb40c9f6b10334a', 70/92),
 ('c2267718faefa8425a25b4bc96d15288', 38/48),
 ('7cc8eee280dde67fb67328535f645a56', 40/50),
 ('3926543c8c89f55aebb66e795de616c5', 92/115),
 ('14958795bf5a737304119fb168a5e441', 90/111),
 ('c1a2999343da2d84effa1679a4a000b6', 218/269),
 ('0ad5b2097d2d7e99ddffbc898f0d4935', 150/182),
 ('a6d97932c543ba2fb32df13249795a18', 142/172),
 ('3ecf920e299e

In [9]:
entities = set()
for _, row in d.iterrows():
    idx = json.loads(row['OUTPUT:output'])
    words = inp.split(' ')
    for start, end in idx:
        e = ' '.join(words[start: end+1])
        entities.add(e)
print(entities)

set()
